# SVD-Hybrid CLIP Light Experiment

This Colab notebook runs the light (4-task) vision experiment for the SVD-Hybrid merging approach combining:
- Task deltas from fine-tuned CLIP ViT-B/32 checkpoints (Cars, EuroSAT, DTD, SUN397)
- SVD-based high/low energy basis split
- 2-bit quantization (placeholder or via TVQ) of low-energy coefficients

## What You Will Do
1. Install dependencies (PyTorch, CLIP, optional TVQ)
2. Clone your repository containing `svd_hybrid_clip/`
3. Provide or simulate fine-tuned task checkpoints
4. Run the light merging pipeline
5. Inspect artifacts (bases, compressed coefficients, merged weights)
6. (Optional) Visualize singular values & reconstruction error

## Before You Start
If you already have real fine-tuned checkpoints, upload them or mount Google Drive. Otherwise you can simulate them for a mechanical test.

## 1. GPU Check

In [ ]:
!nvidia-smi || echo "No GPU detected. Proceeding on CPU may be slow."

## 2. Install Dependencies
If you have a specific CUDA version, adjust the PyTorch index URL. Below uses a common recent CUDA wheel mirror; if it fails, fallback to the default PyPI install.

In [ ]:
%%bash
pip install --upgrade pip
# Try CUDA wheel (adjust if needed). If it fails, rerun without index-url.
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
pip install git+https://github.com/openai/CLIP.git
# Optional: TVQ (if public & accessible)
pip install git+https://github.com/AIM-SKKU/TVQ.git || echo "TVQ optional or unavailable."

## 3. Clone Your Repository
Replace `mgradyn/svd-hybrid-clip` with your actual GitHub <owner>/<repo>. If the repo is private, configure a PAT or use Drive upload instead.

In [ ]:
REPO_URL = "https://github.com/mgradyn/svd-hybrid-clip.git"  # CHANGE if needed
!git clone $REPO_URL
%cd svd-hybrid-clip || echo "Change directory name if your repo differs."

## 4. (Optional) Mount Google Drive for Checkpoints
If you already have fine-tuned CLIP checkpoints in Drive (e.g., in `MyDrive/clip_finetuned/`), copy them into the local `checkpoints/` directory.

Skip this if you'll simulate checkpoints.

In [ ]:
# Uncomment to mount Drive
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p checkpoints
# Example copy command (adjust path):
# !cp /content/drive/MyDrive/clip_finetuned/*.pt checkpoints/ || echo "No real checkpoints copied."

## 5. (Optional) Simulate Fine-Tuned Checkpoints
If you do not have real task-specific checkpoints yet, simulate them by adding small random noise to base CLIP weights. **These are not meaningful for performance metrics**, but let you test the pipeline mechanics.

In [ ]:
import torch
try:
    import clip
except ImportError:
    raise ImportError("CLIP not installed. Re-run installation cell.")

model, preprocess = clip.load("ViT-B/32", device="cpu")
base_sd = {k: v.clone() for k, v in model.state_dict().items() if k.startswith("visual.")}

tasks = ["cars", "eurosat", "dtd", "sun397"]
for t in tasks:
    task_sd = {}
    for k, v in base_sd.items():
        # Small perturbation simulating fine-tune differences
        task_sd[k] = v + 0.01 * torch.randn_like(v)
    torch.save(task_sd, f"checkpoints/{t}_clip_vitb32.pt")

print("Simulated checkpoints created:")
!ls checkpoints

## 6. Verify Checkpoint Files

In [ ]:
import os
print(os.listdir("checkpoints"))

## 7. Patch `TASK_CHECKPOINTS` in `run_light_experiment.py`
Update the script to point to the checkpoint list if the filenames differ.

In [ ]:
import re, json
script_path = "svd_hybrid_clip/run_light_experiment.py"
with open(script_path, "r") as f:
    content = f.read()

new_paths = [
    "checkpoints/cars_clip_vitb32.pt",
    "checkpoints/eurosat_clip_vitb32.pt",
    "checkpoints/dtd_clip_vitb32.pt",
    "checkpoints/sun397_clip_vitb32.pt"
]
new_block = "TASK_CHECKPOINTS = " + json.dumps(new_paths, indent=2)
content = re.sub(r"TASK_CHECKPOINTS\s*=\s*\[.*?\]", new_block, content, flags=re.DOTALL)

with open(script_path, "w") as f:
    f.write(content)
print("Updated TASK_CHECKPOINTS in run_light_experiment.py")

## 8. Run Light SVD-Hybrid Experiment
Outputs:
- Bases (U_high/U_low + meta)
- Compressed per-task coefficient artifacts
- Merged visual backbone state dict

By default uses energy threshold 0.90 and max rank 32.

In [ ]:
!python svd_hybrid_clip/run_light_experiment.py --output-root ./svd_out --data-root ./data --device cuda

## 9. Inspect Artifacts

In [ ]:
import torch
art = torch.load("svd_out/svd_hybrid_clip_artifacts.pt")
print("Layers stored:", list(art["bases"].keys()))
print("Task order:", art["task_order"])
for lname, (_, _, meta) in art["bases"].items():
    print(f"Layer: {lname} | k={meta['k']} | N={meta['N']} | D={meta['D']}")

## 10. (Optional) Visualize Singular Values

In [ ]:
import matplotlib.pyplot as plt
for lname, (_, _, meta) in art["bases"].items():
    sv = meta.get("singular_values")
    if sv is not None:
        plt.figure(figsize=(4,3))
        plt.plot(sv.numpy(), marker='o')
        plt.title(f"Top-k Singular Values: {lname[:35]}...")
        plt.xlabel("Index")
        plt.ylabel("Value")
        plt.show()
        break  # show one example

## 11. (Optional) Compute Reconstruction Errors Per Task & Layer
Shows how well compressed coefficients reconstruct original deltas.

In [ ]:
from svd_hybrid_clip.tvq_adapter import dequantize_low
errors = {}
bases = art["bases"]
artifacts = art["artifacts"] if "artifacts" in art else art.get("artifacts", {})
# If artifacts not stored separately, adjust loading accordingly.
for lname, (U_high, U_low, meta) in bases.items():
    layer_art_list = artifacts.get(lname, [])
    if not layer_art_list:
        continue
    layer_errs = []
    # Original flat deltas were not saved here; approximate by reconstructing each task (for simulation only)
    # If you want true error, modify pipeline to store original flats.
    for art_task in layer_art_list:
        c_high = art_task["c_high_fp16"].float()
        c_low = dequantize_low(art_task["c_low_quant"]).float()
        recon = U_high.float() @ c_high + U_low.float() @ c_low
        # Here we can't compute true error without original delta; skip or assume original ~ recon for simulation.
        layer_errs.append(0.0)
    errors[lname] = layer_errs
print("(Simulated) reconstruction error placeholders per layer:")
for lname, errs in errors.items():
    if errs:
        print(lname, "mean=", sum(errs)/len(errs))

## 12. (Optional) Load Merged Weights Into CLIP Model

In [ ]:
merged_visual_sd = torch.load("svd_out/merged_visual_sd.pt")
model_merged, preprocess2 = clip.load("ViT-B/32", device="cuda" if torch.cuda.is_available() else "cpu")
with torch.no_grad():
    msd = model_merged.state_dict()
    for k, v in merged_visual_sd.items():
        if k in msd:
            msd[k].copy_(v)
print("Merged weights loaded into CLIP model.")

## 13. (Optional) Feature Extraction Demo

In [ ]:
dummy = torch.randn(2, 3, 224, 224, device="cuda" if torch.cuda.is_available() else "cpu")
with torch.no_grad():
    feats = model_merged.encode_image(dummy)
print("Feature shape:", feats.shape)

## 14. Next Steps
- Replace simulated checkpoints with real fine-tuned ones.
- Expand layer coverage (more transformer blocks).
- Integrate real RTVQ (TVQ) quantizer in `tvq_adapter.py`.
- Train a linear probe classifier on merged features per dataset.
- Add weighted averaging of coefficients based on validation performance.
- For large task counts, consider randomized SVD (`torch.svd_lowrank`).

## Done
You have run the light SVD-Hybrid merging pipeline in Colab. Adapt and extend for full experiments.